In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, log_loss
from sklearn.ensemble import RandomForestClassifier
import tabpy_client

In [2]:
df = pd.read_excel('Health_claims.xlsx',sheet_name=1)
df.head()


,Name_of_insurance_co,group_name,policy_no,Start Date,End Date,rel_type,Age,sex,Hospital,Network_city,...,icdcodeSecondLevel,icdcodeThirdLevel,Maternity Remarks,Claim Amt,Final Status,GWP,Incurred amt,PreandPost,PROVIDERTYPE,Claim ID
0,New India Assurance,ABC Ltd,123456,2019-01-01,2019-12-31,Spouse,35,Female,Shanti Nursing Home,Delhi,...,Delivery,Single spontaneous delivery,Normal,14652,Settled,12638.70,13590,Regular,Non network,NaN
1,New India Assurance,ABC Ltd,123456,2019-01-01,2019-12-31,Spouse,25,Female,Shanti Nursing Home,Delhi,...,Respiratory and cardiovascular disorders speci...,Respiratory distress of newborn,NaN,8594,Settled,7248.42,7794,Regular,Non network,NaN
2,New India Assurance,ABC Ltd,123456,2019-01-01,2019-12-31,Spouse,19,Female,Prantik Nursing Home,Krishnagar,...,Delivery,Single delivery by caesarean section,C-Section,49000,Settled,44362.86,47702,Regular,Non network,NaN
3,New India Assurance,ABC Ltd,123456,2019-01-01,2019-12-31,Spouse,62,Female,St Stephens Hospital,Delhi,...,Pregnancy with abortive outcome,Ectopic pregnancy,Other,58514,Settled,44992.47,48379,Regular,Network,NaN
4,New India Assurance,ABC Ltd,123456,2019-01-01,2019-12-31,Employee,90,Female,Specialist Hospital,Bangalore,...,Other diseases of urinary system,Other disorders of urinary system,NaN,28231,Settled,25975.83,27931,Regular,Non network,NaN


In [3]:
df2 = df.filter(['Age','sex','Claim Cost','Claim Cost-Amt. Band','Final Status','Sum Insured','disease_category'],axis = 1)
df2 = df2[df2['disease_category']!='Pregnancy, childbirth and the puerperium']
df2.drop(df2[(df2['disease_category']=='Not mentioned')|(df2['disease_category']=='Not Mentioned')].index,inplace = True)

In [4]:
df2['disease_category'].value_counts()

Certain infectious and parasitic diseases                                                                312
Diseases of the genitourinary system                                                                     305
Diseases of the digestive system                                                                         275
Injury, poisoning and certain other consequences of external causes                                      248
Diseases of the respiratory system                                                                       246
Diseases of the eye and adnexa                                                                           204
Symptoms, signs and abnormal clinical and laboratory findings, not else  where classified                193
Diseases of the circulatory system                                                                       170
Certain conditions originating in the perinatal period                                                   146
Neoplasms          

In [5]:
df2["sex"] = np.where(df2["sex"] == "Female", 0, 1)
df2["Final Status"] = np.where(df2["Final Status"] == "Settled", 1,np.where(df2["Final Status"]=='Outstanding',1,0))
df2['disease_category']=df2['disease_category'].astype('category')

In [6]:
df2["disease_category"] = np.where(df2["disease_category"] == "Certain infectious and parasitic diseases", 1,
                                 np.where(df2["disease_category"] == "Diseases of the genitourinary system", 2,
                                         np.where(df2["disease_category"] == "Diseases of the digestive system", 3,
                                                 np.where(df2["disease_category"] =='Injury, poisoning and certain other consequences of external causes',4,
                                                         np.where(df2["disease_category"] =='Diseases of the respiratory system',5,
                                                                 np.where(df2["disease_category"] =='Diseases of the eye and adnexa',6,
                                                                         np.where(df2["disease_category"] =='Symptoms, signs and abnormal clinical and laboratory findings, not else  where classified',7,
                                                                                 np.where(df2["disease_category"] =='Diseases of the circulatory system',8,
                                                                                         np.where(df2["disease_category"] =='Certain conditions originating in the perinatal period',9,
                                                                                                 np.where(df2["disease_category"] =='Neoplasms',10,
                                                                                                         np.where(df2["disease_category"] =='Diseases of the musculoskeletal system and connective tissue',11,
                                                                                                                 np.where(df2["disease_category"] =='Diseases of the skin and subcutaneous tissue',12,
                                                                                                                         np.where(df2["disease_category"] =='Diseases of the nervous system',13,
                                                                                                                                 np.where(df2["disease_category"] =='Diseases of the ear and mastoid process',14,
                                                                                                                                         np.where(df2["disease_category"] =='Endocrine, nutritional and metabolic diseases',15,
                                                                                                                                                 np.where(df2["disease_category"] =='Diseases of the blood and blood-forming organs and certain   disorders involving the immune mechanism',16,
                                                                                                                                                         np.where(df2["disease_category"] =='Congenital malformations, deformations and chromosomal abnormalities',17,
                                                                                                                                                                 np.where(df2["disease_category"] =='Mental and behavioural disorders',18,19))))))))))))))))))
                                                                                                        

In [7]:
#y = pd.get_dummies(df2.disease_category)
#New_df2=df2.join(y)
df2.drop(columns=['Claim Cost','Claim Cost-Amt. Band','Sum Insured'],axis = 1,inplace=True)
df2.to_csv('health10.csv')
df = pd.read_csv('health10.csv')
df.drop(columns=['Unnamed: 0'],axis = 1,inplace=True)

In [8]:
y_true = df['Final Status']
df.drop(['Final Status'], axis=1, inplace=True)

In [9]:
X_train,X_test, y_train, y_test = train_test_split(df, y_true, stratify=y_true, test_size=0.3)
print("Number of data points in train data :",X_train.shape)
print("Number of data points in test data :",X_test.shape)

Number of data points in train data : (1747, 3)
Number of data points in test data : (750, 3)


In [10]:
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
clf.score(X_train, y_train)
acc_random_forest = round(clf.score(X_train, y_train) * 100, 2)

acc_random_forest

95.88

In [11]:
import tabpy_client
from tabpy.tabpy_tools.client import Client
client = tabpy_client.Client('http://localhost:9004/')

In [13]:
def disease_predictor( _arg1, _arg2, _arg3):
    import pandas as pd
    row = {'Age': _arg1,'sex': _arg2,'disease_category': _arg3}
    test_data = pd.DataFrame(data = row,index=[0])
    predicted = clf.predict_proba(test_data)
    return [prediction[1] for prediction in predicted]
client.deploy('disease_predictor', disease_predictor,'Predicts disease claim', override = True)

    